In [1]:
# Install required packages (clean reinstall to fix circular import issues)
!pip uninstall -y google-generativeai google-ai-generativelanguage
!pip install --no-cache-dir google-generativeai python-dotenv tqdm pandas

Found existing installation: google-generativeai 0.8.5
Uninstalling google-generativeai-0.8.5:
  Successfully uninstalled google-generativeai-0.8.5
Found existing installation: google-ai-generativelanguage 0.6.15
Uninstalling google-ai-generativelanguage-0.6.15:
  Successfully uninstalled google-ai-generativelanguage-0.6.15
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 1.3/1.3 MB 13.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 1.3/1.3 MB 13.7 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
import time
import random
import re
from typing import List, Any
from tqdm import tqdm
import google.generativeai as genai
from collections import deque

# =============================
# Configuration
# =============================
# Prefer using an environment variable; fallback to the hard‑coded value ONLY if set (avoid committing real keys!)
GENAI_API_KEY = os.getenv("GENAI_API_KEY", "")  # <-- replace with your key or set env var
if not GENAI_API_KEY or not GENAI_API_KEY.strip():
    raise RuntimeError("Gemini API key missing. Set GENAI_API_KEY env var or edit the notebook.")

genai.configure(api_key=GENAI_API_KEY)
MODEL_NAME = "gemini-2.5-flash"

# Student level: one of: "excellent", "average", "struggling"
STUDENT_LEVEL = "struggling"  # <-- change to test personas

# Question difficulty: one of: "easy", "medium", "hard"
QUESTION_DIFFICULTY = "hard"  # <-- change to generate different difficulty datasets

# Number of interviewer->candidate turns (including the kickoff interviewer question)
NUM_EXCHANGES = 4  # produces (NUM_EXCHANGES * 2 - 1) total lines in transcript

# =============================
# Difficulty-Based Interviewer Prompts
# =============================
INTERVIEWER_DIFFICULTY_PROMPTS = {
    "easy": """
You are a senior system design interviewer conducting a beginner-friendly interview.
Focus on fundamental concepts and basic architecture (e.g., client-server, simple databases, basic caching).
Ask ONE short focused follow-up (<=15 words) that progresses gradually through: basic components -> simple data flow -> basic scaling.
Use concrete but simple examples. Avoid complex distributed systems concepts.
No bullet lists. No multi-sentence output.
Format strictly: Interviewer: <question>
""".strip(),
    "medium": """
You are a senior system design interviewer conducting a progressive interview.
CRITICAL: Build ONLY on the candidate's previous answers; never restate earlier covered basics.
Ask ONE short focused follow-up (<=15 words) that advances: high-level -> components -> scaling -> trade-offs.
Reference prior statements briefly (e.g., "You mentioned cache; how handle eviction?").
Be concrete (QPS, latency, storage). No bullet lists. No multi-sentence output.
Format strictly: Interviewer: <question>
""".strip(),
    "hard": """
You are a senior system design interviewer conducting an advanced-level interview.
Focus on complex distributed systems challenges: consistency models, partitioning strategies, failure scenarios, performance optimization.
Ask ONE short focused follow-up (<=15 words) that dives deep into: CAP trade-offs -> distributed consensus -> failure handling -> performance bottlenecks -> advanced scaling.
Expect specific numbers (millions of QPS, sub-millisecond latency, petabyte scale) and deep technical reasoning.
Challenge assumptions. No bullet lists. No multi-sentence output.
Format strictly: Interviewer: <question>
""".strip(),
}

# =============================
# Kickoff Question Bank (Multiple Questions per Difficulty)
# =============================
KICKOFF_QUESTION_BANK = {
    "easy": [
        "Interviewer: Design a basic URL shortening service; what's your approach?",
        "Interviewer: Design a simple online bookstore; how would you structure it?",
        "Interviewer: Design a basic chat application; what components do you need?",
        "Interviewer: Design a simple to-do list app; what's your database design?",
        "Interviewer: Design a basic image upload service; how would you store images?",
        "Interviewer: Design a simple blog platform; what's your high-level architecture?",
        "Interviewer: Design a basic voting system; how would you count votes?",
        "Interviewer: Design a simple notification service; what's your approach?",
    ],
    "medium": [
        "Interviewer: Design a scalable URL shortening service; what's your high-level approach?",
        "Interviewer: Design a news feed system for 10M users; what's your architecture?",
        "Interviewer: Design a rate limiter handling 100K requests/sec; how would you build it?",
        "Interviewer: Design a video streaming platform; what are your key components?",
        "Interviewer: Design a ride-sharing service like Uber; what's your matching strategy?",
        "Interviewer: Design a distributed cache system; how do you handle consistency?",
        "Interviewer: Design a real-time leaderboard for 1M gamers; what's your approach?",
        "Interviewer: Design a scalable web crawler; how do you avoid duplicates?",
        "Interviewer: Design a food delivery system; how do you handle real-time tracking?",
        "Interviewer: Design a messaging system like WhatsApp; what's your delivery guarantee?",
    ],
    "hard": [
        "Interviewer: Design a globally distributed URL shortening service handling 100K writes/sec; what's your architecture?",
        "Interviewer: Design YouTube's video recommendation engine at petabyte scale; how do you optimize latency?",
        "Interviewer: Design a distributed transaction system with strict consistency; what's your consensus protocol?",
        "Interviewer: Design Google Search's indexing pipeline processing 100M pages/hour; what's your partitioning strategy?",
        "Interviewer: Design a global payment system handling 1M TPS with <100ms latency; how handle failures?",
        "Interviewer: Design Twitter's timeline at 500M users with <50ms p99 latency; what's your caching strategy?",
        "Interviewer: Design a distributed file system like GFS storing 100PB; how ensure consistency?",
        "Interviewer: Design a real-time bidding system processing 10M bids/sec; what's your optimization approach?",
        "Interviewer: Design Netflix's CDN serving 200M concurrent streams; how minimize buffering globally?",
        "Interviewer: Design a distributed database with multi-region writes; how resolve conflicts?",
    ],
}

# =============================
# System / Persona Prompts
# =============================
STUDENT_PROMPTS = {
    "excellent": """
You are an excellent system design candidate.
Answer in ONE precise sentence (<20 words) using concrete terms (throughput numbers, components, trade-offs) confidently.
No filler. Format: Candidate: <answer>
""".strip(),
    "average": """
You are an average system design candidate.
Answer in ONE sentence (<20 words) mixing some specifics and mild uncertainty ("maybe", "probably").
Brief filler allowed. Format: Candidate: <answer>
""".strip(),
    "struggling": """
You are a struggling system design candidate.
Answer in ONE short sentence (<15 words) with visible uncertainty or partial gaps ("Um", "I guess").
Often simplistic or incomplete. Format: Candidate: <answer>
""".strip(),
}

# =============================
# Hesitation / Speech Patterns
# =============================
HESITATION_PATTERNS = {
    "excellent": {"rate": 0.05, "patterns": ["Right...", "Actually..."]},
    "average": {"rate": 0.35, "patterns": ["Hmm...", "I think...", "Maybe...", "Probably..."]},
    "struggling": {"rate": 0.65, "patterns": ["Um...", "Uh...", "I guess...", "Let me think..."]},
}

# =============================
# Model Factory
# =============================

def make_model(system_prompt: str):
    return genai.GenerativeModel(MODEL_NAME, system_instruction=system_prompt)

interviewer = make_model(INTERVIEWER_DIFFICULTY_PROMPTS[QUESTION_DIFFICULTY])
candidate = make_model(STUDENT_PROMPTS[STUDENT_LEVEL])

# =============================
# Helpers
# =============================
MAX_RETRIES = 5
RETRY_BASE = 1.6

# Rate limiting: keep requests under your free-tier RPM to avoid 429s.
# Default to 9 RPM to leave headroom under the 10 RPM limit.
MAX_RPM = int(os.getenv("GENAI_MAX_RPM", "9"))
_REQUEST_TIMES = deque()  # monotonic timestamps of recent requests


def _rate_limit():
    """Block until we are under the requests-per-minute threshold."""
    now = time.monotonic()
    # Drop timestamps older than 60s window
    while _REQUEST_TIMES and (now - _REQUEST_TIMES[0]) >= 60:
        _REQUEST_TIMES.popleft()
    # If at capacity, wait until earliest timestamp falls out of window
    while len(_REQUEST_TIMES) >= MAX_RPM:
        earliest = _REQUEST_TIMES[0]
        sleep_s = max(0.5, 60 - (time.monotonic() - earliest) + random.random() * 0.3)
        time.sleep(sleep_s)
        # Recompute window after sleeping
        now = time.monotonic()
        while _REQUEST_TIMES and (now - _REQUEST_TIMES[0]) >= 60:
            _REQUEST_TIMES.popleft()
    # Reserve a slot for the outgoing request
    _REQUEST_TIMES.append(time.monotonic())


def call_with_retry(model, contents: Any) -> str:
    delay = 0.7
    for attempt in range(MAX_RETRIES):
        try:
            _rate_limit()
            resp = model.generate_content(contents=contents)
            txt = (resp.text or "").strip()
            if txt:
                return txt
        except Exception as e:  # noqa: BLE001
            msg = str(e)
            # Handle explicit quota/rate-limit signals with a longer sleep
            if (
                "ResourceExhausted" in msg
                or "429" in msg
                or "quota" in msg.lower()
                or "rate" in msg.lower()
            ):
                # Try to extract suggested wait seconds from error message
                m = re.search(r"retry[_ ]?delay[^0-9]*(\d+)", msg, re.I)
                if not m:
                    m = re.search(r"retry in (\d+)", msg, re.I)
                wait_s = int(m.group(1)) if m else 45
                time.sleep(wait_s + random.random() * 0.7)
                # After a long wait, continue without consuming a normal backoff attempt
                continue
            if attempt == MAX_RETRIES - 1:
                raise
            time.sleep(delay + random.random() * 0.2)
            delay *= RETRY_BASE
    return ""


def trim_response_length(text: str, max_sentences: int = 1, max_words: int = 18) -> str:
    # Extract sentences
    sentences = re.split(r'[.!?]+', text)
    sentences = [s.strip() for s in sentences if s.strip()]
    if sentences:
        sentences = sentences[:max_sentences]
        result = '. '.join(sentences)
    else:
        result = text.strip()
    # Ensure terminal punctuation
    if result and result[-1] not in ".!?":
        result += '.'
    # Word trim
    words = result.split()
    if len(words) > max_words:
        result = ' '.join(words[:max_words]) + '...'
    return result


def ensure_prefix(text: str, prefix: str) -> str:
    if not text.startswith(prefix):
        return f"{prefix} {text.lstrip()}"
    return text


def apply_hesitation(body: str) -> str:
    cfg = HESITATION_PATTERNS[STUDENT_LEVEL]
    lower = body.lower()
    starts_hes = any(lower.startswith(p.split('...')[0].strip(' .').lower()) for p in cfg["patterns"])
    # Remove existing hesitation sometimes if too frequent
    if starts_hes and random.random() > cfg["rate"]:
        body = re.sub(r'^(right|actually|hmm+|um+|uh+|i think|maybe|probably|i guess|let me think)[, .\-–—]*', '', body, flags=re.I).lstrip()
    elif not starts_hes and random.random() < cfg["rate"]:
        body = f"{random.choice(cfg['patterns'])} {body}".strip()
    return body


def postprocess_candidate(raw: str) -> str:
    raw = raw.strip()
    raw = raw.replace('\n', ' ')
    # Remove any prefix artifacts
    raw = re.sub(r'^candidate:\s*', '', raw, flags=re.I)
    raw = trim_response_length(raw, max_sentences=1, max_words=18)
    raw = apply_hesitation(raw)
    return ensure_prefix(raw, "Candidate:")


def postprocess_interviewer(raw: str) -> str:
    raw = raw.strip().replace('\n', ' ')
    raw = re.sub(r'^interviewer:\s*', '', raw, flags=re.I)
    raw = trim_response_length(raw, max_sentences=1, max_words=15)
    return ensure_prefix(raw, "Interviewer:")

# =============================
# Conversation Generation
# =============================
# Randomly select a kickoff question from the difficulty-specific question bank
KICKOFF_QUESTION = random.choice(KICKOFF_QUESTION_BANK[QUESTION_DIFFICULTY])

history: List[str] = [KICKOFF_QUESTION]
turn = "candidate"  # next speaker

# We already have first interviewer question. We need NUM_EXCHANGES-1 more interviewer turns.
for _ in range(NUM_EXCHANGES - 1):
    # Candidate answers last interviewer question
    last_question = history[-1]
    cand_raw = call_with_retry(candidate, last_question)
    cand_msg = postprocess_candidate(cand_raw)
    history.append(cand_msg)

    # Interviewer asks follow-up using full transcript context
    context = "\n".join(history)
    int_raw = call_with_retry(interviewer, context)
    int_msg = postprocess_interviewer(int_raw)
    history.append(int_msg)

# =============================
# Output Transcript
# =============================
print("\n".join(history))
print(f"\n--- Generated with Difficulty: {QUESTION_DIFFICULTY.upper()} | Student Level: {STUDENT_LEVEL.upper()} ---")


c:\Users\prana\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Interviewer: Design Twitter's timeline at 500M users with <50ms p99 latency; what's your caching strategy?
Candidate: I guess just cache the latest tweets in Redis or something.
Interviewer: How do you maintain cache consistency for 500M dynamic timelines.
Candidate: Um, I guess we could use a Time-To-Live for cache entries.
Interviewer: How does TTL alone guarantee acceptable timeline freshness for 500M users.
Candidate: Um, TTL just means data expires after a set time, I guess.
Interviewer: How do you guarantee users see new tweets promptly with just TTL.

--- Generated with Difficulty: HARD | Student Level: STRUGGLING ---


In [3]:
# =============================
# Generate 10 Diverse Datasets
# =============================

# Define 10 different configurations for maximum diversity
DATASET_CONFIGS = [
    {"student": "excellent", "difficulty": "hard", "exchanges": 8},      # 1: Expert on complex problems (longest)
    {"student": "struggling", "difficulty": "easy", "exchanges": 4},     # 2: Beginner on simple topics (shortest)
    {"student": "average", "difficulty": "medium", "exchanges": 6},      # 3: Typical mid-level interview
    {"student": "excellent", "difficulty": "easy", "exchanges": 6},      # 4: Expert breezing through basics
    {"student": "struggling", "difficulty": "hard", "exchanges": 6},     # 5: Weak candidate struggling with complexity
    {"student": "average", "difficulty": "easy", "exchanges": 8},        # 6: Average student on simple topics (detailed)
    {"student": "excellent", "difficulty": "medium", "exchanges": 4},    # 7: Expert giving quick scalable answers
    {"student": "struggling", "difficulty": "medium", "exchanges": 8},   # 8: Struggling candidate on moderate problems
    {"student": "average", "difficulty": "hard", "exchanges": 4},        # 9: Average student facing tough questions
    {"student": "excellent", "difficulty": "hard", "exchanges": 6},      # 10: Expert on complex systems (balanced)
]

def generate_single_dataset(student_level: str, difficulty: str, num_exchanges: int) -> str:
    """Generate a single interview transcript with given configuration."""
    
    # Create models for this specific configuration
    interviewer_model = make_model(INTERVIEWER_DIFFICULTY_PROMPTS[difficulty])
    candidate_model = make_model(STUDENT_PROMPTS[student_level])
    
    # Select random kickoff question
    kickoff = random.choice(KICKOFF_QUESTION_BANK[difficulty])
    
    # Initialize conversation history
    history = [kickoff]
    
    # Generate conversation
    for _ in range(num_exchanges - 1):
        # Candidate answers
        last_question = history[-1]
        cand_raw = call_with_retry(candidate_model, last_question)
        
        # Apply hesitation with current student level context
        cand_raw = cand_raw.strip().replace('\n', ' ')
        cand_raw = re.sub(r'^candidate:\s*', '', cand_raw, flags=re.I)
        cand_raw = trim_response_length(cand_raw, max_sentences=1, max_words=18)
        
        # Apply hesitation patterns
        cfg = HESITATION_PATTERNS[student_level]
        lower = cand_raw.lower()
        starts_hes = any(lower.startswith(p.split('...')[0].strip(' .').lower()) for p in cfg["patterns"])
        if starts_hes and random.random() > cfg["rate"]:
            cand_raw = re.sub(r'^(right|actually|hmm+|um+|uh+|i think|maybe|probably|i guess|let me think)[, .\-–—]*', '', cand_raw, flags=re.I).lstrip()
        elif not starts_hes and random.random() < cfg["rate"]:
            cand_raw = f"{random.choice(cfg['patterns'])} {cand_raw}".strip()
        
        cand_msg = ensure_prefix(cand_raw, "Candidate:")
        history.append(cand_msg)
        
        # Interviewer asks follow-up
        context = "\n".join(history)
        int_raw = call_with_retry(interviewer_model, context)
        int_raw = int_raw.strip().replace('\n', ' ')
        int_raw = re.sub(r'^interviewer:\s*', '', int_raw, flags=re.I)
        int_raw = trim_response_length(int_raw, max_sentences=1, max_words=15)
        int_msg = ensure_prefix(int_raw, "Interviewer:")
        history.append(int_msg)
    
    # Format the transcript
    transcript = "\n".join(history)
    metadata = f"\n--- Difficulty: {difficulty.upper()} | Student Level: {student_level.upper()} | Exchanges: {num_exchanges} ---"
    
    return transcript + metadata

# Generate all 10 datasets
print("Starting dataset generation...\n")
all_datasets = []

for idx, config in enumerate(tqdm(DATASET_CONFIGS, desc="Generating datasets"), start=1):
    print(f"\nGenerating Dataset {idx}: {config['student']} + {config['difficulty']} + {config['exchanges']} exchanges")
    
    dataset = generate_single_dataset(
        student_level=config["student"],
        difficulty=config["difficulty"],
        num_exchanges=config["exchanges"]
    )
    
    all_datasets.append(f"========== DATASET {idx} ==========\n{dataset}\n")
    print(f"✓ Dataset {idx} generated successfully!")

# Save all datasets to a single file
output_file = "all_datasets.txt"
with open(output_file, 'w', encoding='utf-8') as f:
    f.write("\n\n".join(all_datasets))

print(f"\n{'='*60}")
print(f"✅ SUCCESS! All 10 datasets generated and saved to: {output_file}")
print(f"{'='*60}")
print("\nDataset Summary:")
for idx, config in enumerate(DATASET_CONFIGS, start=1):
    print(f"  {idx}. {config['student'].capitalize():12} | {config['difficulty'].capitalize():6} | {config['exchanges']} exchanges")
print(f"\nTotal file size: {os.path.getsize(output_file) / 1024:.2f} KB")


Starting dataset generation...



Generating datasets:   0%|          | 0/10 [00:00<?, ?it/s]


Generating Dataset 1: excellent + hard + 8 exchanges


Generating datasets:   0%|          | 0/10 [00:01<?, ?it/s]



KeyboardInterrupt: 

In [4]:
# =============================
# Generate 10 JSON Datasets (exchanges >= 6)
# =============================
import json
from datetime import datetime

# Configurable range for exchanges (must be >= 6 as requested)
MIN_EXCHANGES, MAX_EXCHANGES = 6, 10
SAMPLE_COUNT = 5 # Generate 10 samples per run
OUTPUT_JSON = "all_datasets.json"

LEVEL_CHOICES = ["excellent", "average", "struggling"]
DIFFICULTY_CHOICES = ["easy", "medium", "hard"]


def to_turn(s: str) -> dict:
    """Convert a prefixed line (e.g., 'Interviewer: ...') into a structured turn."""
    s = s.strip()
    low = s.lower()
    if low.startswith("interviewer:"):
        return {"speaker": "Interviewer", "text": s.split(":", 1)[1].strip()}
    if low.startswith("candidate:"):
        return {"speaker": "Candidate", "text": s.split(":", 1)[1].strip()}
    return {"speaker": "Unknown", "text": s}


def generate_dataset_dict(student_level: str, difficulty: str, num_exchanges: int, sample_id: int) -> dict:
    """Generate one dataset as a structured dict, without modifying global STUDENT_LEVEL."""
    interviewer_model = make_model(INTERVIEWER_DIFFICULTY_PROMPTS[difficulty])
    candidate_model = make_model(STUDENT_PROMPTS[student_level])

    # Kickoff
    kickoff_full = random.choice(KICKOFF_QUESTION_BANK[difficulty])
    kickoff_text = kickoff_full.split(":", 1)[1].strip() if ":" in kickoff_full else kickoff_full

    history: List[str] = [kickoff_full]

    # Conversation generation (num_exchanges includes the kickoff interviewer turn)
    for _ in range(num_exchanges - 1):
        # Candidate answers
        last_question = history[-1]
        cand_raw = call_with_retry(candidate_model, last_question)
        cand_raw = cand_raw.strip().replace("\n", " ")
        cand_raw = re.sub(r'^candidate:\s*', '', cand_raw, flags=re.I)
        cand_raw = trim_response_length(cand_raw, max_sentences=1, max_words=18)

        # Apply hesitation based on this dataset's student level (not global STUDENT_LEVEL)
        cfg = HESITATION_PATTERNS[student_level]
        lower = cand_raw.lower()
        starts_hes = any(lower.startswith(p.split('...')[0].strip(' .').lower()) for p in cfg["patterns"])
        if starts_hes and random.random() > cfg["rate"]:
            cand_raw = re.sub(r'^(right|actually|hmm+|um+|uh+|i think|maybe|probably|i guess|let me think)[, .\-–—]*', '', cand_raw, flags=re.I).lstrip()
        elif not starts_hes and random.random() < cfg["rate"]:
            cand_raw = f"{random.choice(cfg['patterns'])} {cand_raw}".strip()

        cand_msg = ensure_prefix(cand_raw, "Candidate:")
        history.append(cand_msg)

        # Interviewer follow-up
        context = "\n".join(history)
        int_raw = call_with_retry(interviewer_model, context)
        int_raw = int_raw.strip().replace('\n', ' ')
        int_raw = re.sub(r'^interviewer:\s*', '', int_raw, flags=re.I)
        int_raw = trim_response_length(int_raw, max_sentences=1, max_words=15)
        int_msg = ensure_prefix(int_raw, "Interviewer:")
        history.append(int_msg)

    # Build structured turns
    turns = [to_turn(line) for line in history]

    return {
        "id": sample_id,
        "student_level": student_level,
        "difficulty": difficulty,
        "exchanges": num_exchanges,
        "kickoff_question": kickoff_text,
        "turns": turns,
        "meta": {
            "model": MODEL_NAME,
            "generated_at": datetime.utcnow().isoformat() + "Z",
        },
    }


print(f"Starting JSON dataset generation for {SAMPLE_COUNT} samples (each with exchanges between {MIN_EXCHANGES} and {MAX_EXCHANGES})...\n")

# Check if file exists and load existing data
existing_datasets = []
if os.path.exists(OUTPUT_JSON):
    try:
        with open(OUTPUT_JSON, "r", encoding="utf-8") as f:
            existing_datasets = json.load(f)
        print(f"📂 Found existing file with {len(existing_datasets)} samples. Will append new ones.\n")
    except:
        print(f"⚠️  Existing file found but couldn't read it. Starting fresh.\n")

# Determine starting ID
start_id = len(existing_datasets) + 1

datasets = []
for i in tqdm(range(start_id, start_id + SAMPLE_COUNT), desc=f"Generating {SAMPLE_COUNT} JSON datasets"):
    level = random.choice(LEVEL_CHOICES)
    diff = random.choice(DIFFICULTY_CHOICES)
    exchanges = random.randint(MIN_EXCHANGES, MAX_EXCHANGES)

    ds = generate_dataset_dict(level, diff, exchanges, i)
    datasets.append(ds)

# Combine with existing data
all_datasets = existing_datasets + datasets

# Save as a single JSON array file
with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
    json.dump(all_datasets, f, ensure_ascii=False, indent=2)

print(f"\n{'='*60}")
print(f"✅ SUCCESS! Generated {SAMPLE_COUNT} new datasets")
print(f"📊 Total datasets in file: {len(all_datasets)}")
print(f"💾 Saved to: {OUTPUT_JSON}")
print(f"{'='*60}")


Starting JSON dataset generation for 5 samples (each with exchanges between 6 and 10)...

📂 Found existing file with 45 samples. Will append new ones.



Generating 5 JSON datasets:   0%|          | 0/5 [00:00<?, ?it/s]C:\Users\prana\AppData\Local\Temp\ipykernel_2932\3978103227.py:80: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "generated_at": datetime.utcnow().isoformat() + "Z",
Generating 5 JSON datasets:  20%|██        | 1/5 [01:49<07:16, 109.02s/it]C:\Users\prana\AppData\Local\Temp\ipykernel_2932\3978103227.py:80: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "generated_at": datetime.utcnow().isoformat() + "Z",
Generating 5 JSON datasets: 100%|██████████| 5/5 [08:30<00:00, 102.10s/it]


✅ SUCCESS! Generated 5 new datasets
📊 Total datasets in file: 50
💾 Saved to: all_datasets.json
